<a id='top'></a>

# TransferMarkt Player Bio and Status Web Scraping
##### Notebook to scrape raw data from [TransferMarkt](https://www.transfermarkt.co.uk/) using [Beautifulsoup](https://pypi.org/project/beautifulsoup4/) and the [Tyrone Mings web scraper](https://github.com/FCrSTATS/tyrone_mings) by [FCrSTATS](https://twitter.com/FC_rstats).

### By [Edd Webster](https://www.twitter.com/eddwebster)
Notebook first written: 13/09/2020<br>
Notebook last updated: 30/08/2021

![title](../../img/transfermarkt-logo-banner.png)

___

<a id='sectionintro'></a>

## <a id='import_libraries'>Introduction</a>
This notebook scrapes data from [TransferMarkt](https://www.transfermarkt.co.uk/) using [Beautifulsoup](https://pypi.org/project/beautifulsoup4/) and the [Tyrone Mings web scraper](https://github.com/FCrSTATS/tyrone_mings) by [FCrSTATS](https://twitter.com/FC_rstats). This landed data is then manipulated as DataFrames using [pandas](http://pandas.pydata.org/).

For more information about this notebook and the author, I'm available through all the following channels:
*    [eddwebster.com](https://www.eddwebster.com/);
*    edd.j.webster@gmail.com;
*    [@eddwebster](https://www.twitter.com/eddwebster);
*    [linkedin.com/in/eddwebster](https://www.linkedin.com/in/eddwebster/);
*    [github/eddwebster](https://github.com/eddwebster/);
*    [public.tableau.com/profile/edd.webster](https://public.tableau.com/profile/edd.webster);
*    [kaggle.com/eddwebster](https://www.kaggle.com/eddwebster); and
*    [hackerrank.com/eddwebster](https://www.hackerrank.com/eddwebster).

![title](../../img/fifa21eddwebsterbanner.png)

The accompanying GitHub repository for this notebook can be found [here](https://github.com/eddwebster/football_analytics) and a static version of this notebook can be found [here](https://nbviewer.jupyter.org/github/eddwebster/football_analytics/blob/master/notebooks/A%29%20Web%20Scraping/TransferMarkt%20Web%20Scraping%20and%20Parsing.ipynb).

___

<a id='sectioncontents'></a>

## <a id='notebook_contents'>Notebook Contents</a>
1.    [Notebook Dependencies](#section1)<br>
2.    [Project Brief](#section2)<br>
3.    [Data Sources](#section3)<br>
      1.    [Introduction](#section3.1)<br>
      2.    [Data Dictionary](#section3.2)<br>
      3.    [Scraping the Data](#section3.3)<br>  
4.    [Summary](#section4)<br>
5.    [Next Steps](#section5)<br>
6.    [Bibliography](#section6)<br>

___

<a id='section1'></a>

## <a id='#section1'>1. Notebook Dependencies</a>
This notebook was written using [Python 3](https://docs.python.org/3.7/) and requires the following libraries:
*    [`Jupyter notebooks`](https://jupyter.org/) for this notebook environment with which this project is presented;
*    [`NumPy`](http://www.numpy.org/) for multidimensional array computing;
*    [`pandas`](http://pandas.pydata.org/) for data analysis and manipulation; and
*    [`Beautifulsoup`](https://pypi.org/project/beautifulsoup4/) for web scraping.

All packages used for this notebook except for BeautifulSoup can be obtained by downloading and installing the [Conda](https://anaconda.org/anaconda/conda) distribution, available on all platforms (Windows, Linux and Mac OSX). Step-by-step guides on how to install Anaconda can be found for Windows [here](https://medium.com/@GalarnykMichael/install-python-on-windows-anaconda-c63c7c3d1444) and Mac [here](https://medium.com/@GalarnykMichael/install-python-on-mac-anaconda-ccd9f2014072), as well as in the Anaconda documentation itself [here](https://docs.anaconda.com/anaconda/install/).

### Import Libraries and Modules

In [1]:
# Python ≥3.5 (ideally)
import platform
import sys, getopt
assert sys.version_info >= (3, 5)
import csv

# Import Dependencies
%matplotlib inline

# Math Operations
import numpy as np
from math import pi

# Datetime
import datetime
from datetime import date
import time

# Data Preprocessing
import pandas as pd
import os
import re
import random
from io import BytesIO
from pathlib import Path

# Reading directories
import glob
import os
from os.path import basename

# Flatten lists
from functools import reduce

# Working with JSON
import json
from pandas.io.json import json_normalize

# Web Scraping
import requests
from bs4 import BeautifulSoup
import re

# APIs
from tyrone_mings import * 

# Fuzzy Matching - Record Linkage
import recordlinkage
import jellyfish
import numexpr as ne

# Data Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno as msno

# Progress Bar
from tqdm import tqdm

# Display in Jupyter
from IPython.display import Image, YouTubeVideo
from IPython.core.display import HTML

# Ignore Warnings
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

print('Setup Complete')

Setup Complete


In [2]:
# Python / module versions used here for reference
print('Python: {}'.format(platform.python_version()))
print('NumPy: {}'.format(np.__version__))
print('pandas: {}'.format(pd.__version__))
print('matplotlib: {}'.format(mpl.__version__))

Python: 3.7.6
NumPy: 1.20.3
pandas: 1.3.2
matplotlib: 3.4.2


### Defined Filepaths

In [3]:
# Set up initial paths to subfolders
base_dir = os.path.join('..', '..')
data_dir = os.path.join(base_dir, 'data')
data_dir_tm = os.path.join(base_dir, 'data', 'tm')
img_dir = os.path.join(base_dir, 'img')
fig_dir = os.path.join(base_dir, 'img', 'fig')
video_dir = os.path.join(base_dir, 'video')

### Defined Variables

In [4]:
# Defined variables

## Define today's date
today = datetime.datetime.now().strftime('%d/%m/%Y').replace('/', '')

### Defined Lists

In [5]:
# Defined Lists

## Define list of league codes
df_leagues = pd.read_csv(data_dir_tm + '/reference/tm_leagues_comps.csv')
lst_league_codes = df_leagues['league_code'].to_numpy().tolist()

## Define list of 'Big 5' European Leagues and MLS codes
lst_big5_mls_league_codes = ['GB1', 'FR1', 'L1', 'IT1', 'ES1', 'MLS1']

## Define seasons to scrape
lst_seasons = ['2016', '2017', '2018', '2019', '2020', '2021']

## Define list of folders
lst_folders = ['raw', 'engineered', 'reference']

## Define list of folders
lst_data_types = ['bio', 'joined', 'historical_market_values', 'performance', 'player_urls', 'status', 'transfer_history', 'transfers']

### Defined Dictionaries

In [6]:
### Defined Dictionaries

## Season
dict_seasons = {'2000': '2000/2001',
                '2001': '2001/2002',
                '2002': '2002/2003',
                '2003': '2003/2004',
                '2004': '2004/2005',
                '2005': '2005/2006',
                '2006': '2006/2007',
                '2007': '2007/2008',
                '2008': '2008/2009',
                '2009': '2009/2010',
                '2010': '2010/2011',
                '2011': '2011/2012',
                '2012': '2012/2013',
                '2013': '2013/2014',
                '2014': '2014/2015',
                '2015': '2015/2016',
                '2016': '2016/2017',
                '2017': '2017/2018',
                '2018': '2018/2019',
                '2019': '2019/2020',
                '2020': '2020/2021',
                '2021': '2021/2022',
                '2022': '2022/2023',
                '2023': '2023/2024',
               }

### Custom Functions (Scrapers)
Here, some of the functions from the Tyrone Mings library have been directly into this notebook due to bug fix. Once the pull request made is accepted, this section can be removed - EW 29/12/2020.

In [7]:
import js2xml
import datetime
import warnings
from lxml import etree

In [8]:
# The following code is pasted from the Tyrone Mings library written by FC.rStats

def get_souped_page(page_url):
    '''
    In order not to be blocked for scraping its import to request pages with
    some settings to look more like an actual browser.

    this function takes a page_url from https://www.transfermarkt.com and returns the
    souped page
    '''
    headers = {'User-Agent':
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    pageTree = requests.get(page_url, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    return(pageSoup)



def remove_youth(team_string):
    '''
    There are many variations of each club name to identify each squad within
    that club.

    this function takes a team string and returns the team string without youth
    variations
    '''
    team_string = team_string.replace("U16", "")
    team_string = team_string.replace("U17", "")
    team_string = team_string.replace("U18", "")
    team_string = team_string.replace("U19", "")
    team_string = team_string.replace("U20", "")
    team_string = team_string.replace("U21", "")
    team_string = team_string.replace("U22", "")
    team_string = team_string.replace("U23", "")
    team_string = team_string.replace("u16", "")
    team_string = team_string.replace("u17", "")
    team_string = team_string.replace("u18", "")
    team_string = team_string.replace("u19", "")
    team_string = team_string.replace("u20", "")
    team_string = team_string.replace("u21", "")
    team_string = team_string.replace("u22", "")
    team_string = team_string.replace("u23", "")
    team_string = team_string.replace("ii", "")
    team_string = team_string.replace("Youth", "")
    team_string = team_string.replace("jugend", "")
    team_string = team_string.strip()
    return(team_string)


def calculate_age_at_transfer(born, transfer_date):
    '''
    Calculate the age between the date of transfer and the date of birth of the
    player
    '''
    return transfer_date.year - born.year - ((transfer_date.month, transfer_date.day) < (born.month, born.day))


def calculate_age(born, competition_start):
    '''
    Calculate the age between the start date of a competition and the date of birth of the
    player
    '''
    return(competition_start.year - born.year - ((competition_start.month, competition_start.day) < (born.month, born.day)))


def stringify_children(node):
    '''
    a helper to convert the market value chart data into strings
    '''
    s = node.text
    if s is None:
        s = ''
    for child in node:
        s += etree.tostring(child, encoding='unicode')
    return s


def month_to_number(month_string):
    '''
    a helper to change month abbreviations to month numbers
    '''
    if month_string == "Jan":
        return(1)
    elif month_string == "Feb":
        return(2)
    elif month_string == "Mar":
        return(3)
    elif month_string == "Apr":
        return(4)
    elif month_string == "May":
        return(5)
    elif month_string == "Jun":
        return(6)
    elif month_string == "Jul":
        return(7)
    elif month_string == "Aug":
        return(8)
    elif month_string == "Sep":
        return(9)
    elif month_string == "Oct":
        return(10)
    elif month_string == "Nov":
        return(11)
    elif month_string == "Dec":
        return(12)


In [9]:
def get_club_urls_from_league_page(club_url):
    '''
    From a league page such as :
    https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
    retrived the url links for all clubs
    '''

    league_base_page = get_souped_page(club_url)

    club_urls = []
    for row in league_base_page.find_all('table', 'items')[0].select('tr'):
        for item in row.find_all('td', 'hauptlink'):
            try:
                link = item.select('a')[0]['href']
                if link != None:
                    if len(link) > 0:
                        club_urls.append("https://www.transfermarkt.com" + link)
            except:
                pass

    return(list(set(club_urls)))



def get_player_urls_from_club_page(club_url):
    '''
    From a club page such as :
    https://www.transfermarkt.com/manchester-united/startseite/verein/985/saison_id/2019
    retrived the url links for all players
    '''
    club_base_page = get_souped_page(club_url)

    player_urls = []
    for row in club_base_page.find_all('table', 'items')[0].select('tr'):
        for item in row.find_all('td', 'hauptlink'):
            try:
                link = item.select('a')[0]['href']
                if link != None:
                    if len(link) > 0:
                        player_urls.append("https://www.transfermarkt.com" + link)
            except:
                pass

    return(list(set(player_urls)))



def get_player_urls_from_league_page(league_url, verbose = False):
    '''
    From a league page such as :
    https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
    retrived the url links for all players from all clubs

    if you want to check on progress chhange verbose to True
    '''
    players = []

    clubs = get_club_urls_from_league_page(league_url)
    for c in clubs:
        players = players +  get_player_urls_from_club_page(c)
        if verbose:
            print(c.split("/")[3].replace("-", " "), "players added")
    return(players)

def get_league_mean_player_value_for_season(league_url, season):
    '''
    From a league page such as :
    https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
    retrived the mean Transfermarkt player valuation for the league in a season
    '''
    # change url to include season
    league_url = league_url + '/plus/?saison_id=' + season
    # load table as html
    league_base_page = get_souped_page(league_url)
    div = league_base_page.findAll('div', {'class':'responsive-table'})[0]
    data_table = div.find('table')
    # return dummy if values are not available
    if data_table.find('tr').findAll('th')[-1].text !='ø MV':
      return 0
    # get table body
    dt_body = data_table.find('tbody')
    # for every row, get value of the last column and translate into an integer
    results = []
    for row in dt_body.findAll('tr'):
      val = row.findAll('td')[-1].text
      if val == '-':
        val = '0'
      else:
        val = val.replace('€','').replace('m','0000').replace('Th','000').replace('.','')
      results.append(int(val))
    return np.mean(results)


In [10]:
def bio_player_pull(pageSoup, player_id):

############# ADD SECONDARY POSITION INFO #################### #TODO
    ## base info
    player_name = pageSoup.select('h1')[0].get_text().lower()

    DOB = None
    POB = None
    COB = None
    position = None
    age = None
    height = None
    foot = None
    citizenship = None
    second_citizenship = None

    for row in pageSoup.select('tr'):
        try:

            if row.select('th')[0].get_text().strip() == "Date of birth:":
                DOB = row.select('td')[0].get_text().strip()

            # COB = None
            if row.select('th')[0].get_text().strip() == "Place of birth:":
                POB = row.select('td')[0].get_text().strip()
                COB = row.select('td')[0].select('img')[0]['alt']
                
            if row.select('th')[0].get_text().strip() == "Position:":
                position = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Age:":
                age = int(row.select('td')[0].get_text().strip())

            if row.select('th')[0].get_text().strip() == "Height:":
                height = int(float(row.select('td')[0].get_text().strip().replace('m', '').replace(',', '.').strip())*100)

            if row.select('th')[0].get_text().strip() == "Foot:":
                foot = row.select('td')[0].get_text().strip()

        except:
            pass

    if COB == None:
        for row in pageSoup.select('tr'):
            try:
                if row.select('th')[0].get_text().strip() == "Citizenship:":
                    COB = row.select('td')[0].get_text().strip()
            except:
                pass

    for row in pageSoup.select('tr'):
        try:
            if row.select('th')[0].get_text().strip() == "Citizenship:":
                no_of_citizenships = len(row.select('td')[0].select('img'))
                if no_of_citizenships < 2:
                    citizenship = row.select('td')[0].select('img')[0]['alt']
                if no_of_citizenships > 1:
                    second_citizenship = row.select('td')[0].select('img')[1]['alt']
                else:
                    second_citizenship = None
        except:
            pass


    if DOB != None:
        DOB = DOB.replace(" Happy Birthday", "")
        year_of_birth = int(DOB[len(DOB)-4:])
        month_of_birth = month_to_number(DOB.split(" ")[0])
        day_of_birth = int(DOB.split(" ")[1].split(",")[0])
        DOB = datetime.date(year_of_birth, month_of_birth, day_of_birth)

    else:
        year_of_birth = None
        month_of_birth = None
        day_of_birth = None

    biodict = {
        "player_id": player_id,
        "player_name": player_name,
        "day_of_birth": day_of_birth,
        "month_of_birth": month_of_birth,
        "year_of_birth": year_of_birth,
        "pob": POB,
        "cob": COB,
        "dob": DOB,
        "position": position,
        "height": height,
        "foot": foot,
        "citizenship": citizenship,
        "second_citizenship": second_citizenship
    }

    return(biodict)


def current_football_bio_player_pull(pageSoup, player_id):

    if len(pageSoup.select('div.dataRibbonRIP')) > 0:
        current_club = "dead"
        current_club_country = "NA"
    else:
        current_club = pageSoup.select('div.dataZusatzImage')[0].select('img')[0].get('alt').lower()
        if current_club in ["retired", "without club", "unknown"]:
            current_club_country = "NA"
        else:
            current_club_country = pageSoup.select('div.dataZusatzDaten')[0].select('img')[0].get('alt').lower()


    if len(pageSoup.select('div.dataMarktwert')) == 0:
        market_value = 0

    else:
        market_value = pageSoup.select('div.dataMarktwert')[0].get_text().split("Last update")[0].strip()

        if "m" in market_value:
            market_value = int( float(market_value.strip().replace("â‚¬", "").replace("€","").replace("m","")) * 1000000 )
        elif "Th." in market_value:
            market_value = int(market_value.strip().replace("â‚¬", "").replace("€","").replace("Th.","")) * 1000
        elif "-":
            market_value = 0

    joined = None
    contract_expires = None
    contract_option = None
    on_loan_from = None
    on_loan_from_url = None
    on_loan_from_country = None
    loan_contract_expiry = None
    player_agent = None

    for row in pageSoup.select('tr'):
        try:

            if row.select('th')[0].get_text().strip() == "Joined:":
                joined = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Contract expires:":
                contract_expires = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Contract option:":
                contract_option = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "On loan from:":
                on_loan_from = row.select('td')[0].get_text().strip().lower()
                on_loan_from_url = "https://www.transfermarkt.com" + row.select('td')[0].select('a')[0]['href']

            if row.select('th')[0].get_text().strip() == "Contract there expires:":
                loan_contract_expiry = row.select('td')[0].get_text().strip()

            if row.select('th')[0].get_text().strip() == "Player agent:":
                player_agent = row.select('td')[0].get_text().strip()
        except:
            pass

    if joined != None:
        year_joined = int(joined[len(joined)-4:])
        month_joined = month_to_number(joined.split(" ")[0])
        day_joined = int(joined.split(" ")[1].split(",")[0])
        joined = datetime.date(year_joined, month_joined, day_joined)

    if contract_expires != None:
        if contract_expires != "-":
            year_expired = int(contract_expires[len(contract_expires)-4:])
            month_expired = month_to_number(contract_expires.split(" ")[0])
            day_expired = int(contract_expires.split(" ")[1].split(",")[0])
            contract_expires = datetime.date(year_expired, month_expired, day_expired)
        else:
            contract_expires = None

    if loan_contract_expiry != None:
        if loan_contract_expiry != "-":
            year_expires = int(loan_contract_expiry[len(loan_contract_expiry)-4:])
            month_expires = month_to_number(loan_contract_expiry.split(" ")[0])
            day_expires = int(loan_contract_expiry.split(" ")[1].split(",")[0])
            loan_contract_expiry = datetime.date(year_expires, month_expires, day_expires)
        else:
            loan_contract_expiry = None

    if loan_contract_expiry != None:
        loan_contract_expiry,contract_expires = contract_expires,loan_contract_expiry

    if on_loan_from != None:
        temp_soup = get_souped_page(on_loan_from_url)
        on_loan_from_country = temp_soup.select('div.dataZusatzDaten')[0].select('img')[0]['alt'].lower()

    statusdict = {
        "player_id": player_id,
        "current_club": current_club,
        "current_club_country": current_club_country,
        "market_value": market_value,
        "joined": joined,
        "contract_expires": contract_expires,
        "contract_option": contract_option,
        "on_loan_from": on_loan_from,
        "on_loan_from_country": on_loan_from_country,
        "loan_contract_expiry": loan_contract_expiry,
        "player_agent": player_agent}

    return(statusdict)

def transfer_history_pull(pageSoup, player_id):


    transfered_from = []
    transferred_to = []
    market_values = []
    transfer_fees = []
    transfer_dates = []
    transfer_season = []
    country_from = []
    country_to = []

    market_values_value = None
#         print("player transfer history")
    not_first_row = True
    for box in pageSoup.select('div.box.transferhistorie'):
        if not_first_row:
            not_first_row = True

            for row in box.select('tr')[2:]:
                try:
                    transfered_from_value = row.select('td')[4].select('a')[0].get('href').split("/")[1].replace("-", " ")
#                 if transfered_from_value == ""
                    transfered_from.append(transfered_from_value)
                except:
                    pass

            for row in box.select('tr')[2:]:
                try:
                    transferred_to_value = row.select('td')[8].select('a')[0].get('href').split("/")[1].replace("-", " ")
                    transferred_to.append(transferred_to_value)
                except:
                    pass

            for row in box.select('tr')[2:]:
                try:
                    market_values_value = row.select('td.zelle-mw')[0].get_text()#.select('img')[0].get('alt')

                    if "m" in market_values_value:
                        market_values_value = int( float(market_values_value.replace("€","").replace("m","")) * 1000000 )
                    elif "k" in market_values_value:
                        market_values_value = int(market_values_value.replace("€","").replace("k","")) * 1000
                    elif "-":
                        market_values_value = 0



                    market_values.append(market_values_value)
                except:
                    pass


        ## grab COUNTRY TO
            for row in box.select('tr')[2:]:
                try:
                    no_images = len(row.select('td')[7].select('img'))
                    if no_images > 0:
                        country_to.append(row.select('td')[7].select('img')[0].get('title').lower())
                    else:
                        country_to.append("no country")
                except:
                    pass


        ## grab COUNTRY FROM
            for row in box.select('tr')[2:]:
                try:
                    no_images = len(row.select('td')[3].select('img'))
                    if no_images > 0:
                        country_from.append(row.select('td')[3].select('img')[0].get('title').lower())
                    else:
                        country_from.append("no country")
                except:
                    pass


        ## grab TRANSFER FEE
        for row in box.select('tr')[2:]:
            try:
                transfer_fees_raw = row.select('td.zelle-abloese')[0].get_text()
                transfer_fees.append(transfer_fees_raw)
            except:
                pass


        ## grab TRANSFER DATE
        for row in box.select('tr')[1:]:
            try:
                date_raw = ""
                date_raw = row.select('td.show-for-small')[0].get_text().strip()#.get_text())#.select('img')[0].get('alt')
                if 'Date' in date_raw:

                    date_raw = date_raw.split(": ")[1]
                    year_of_transfer = int(date_raw[len(date_raw)-4:])
                    month_of_transfer = month_to_number(date_raw.split(" ")[0])
                    day_of_transfer = int(date_raw.split(" ")[1].split(",")[0])
                    transfer_date = datetime.date(year_of_transfer, month_of_transfer, day_of_transfer)
                    transfer_dates.append(transfer_date)
            except:
                pass

        ## grab SEASON
        for row in box.select('tr')[1:]:
            try:
                season_raw = row.select('td')[0].get_text().strip()
                if "Date" in season_raw:
                    pass
                else:
                    if "Total" in season_raw:
                        pass
                    else:
                        if "/" in season_raw:
                            transfer_season.append(season_raw.strip())
            except:
                pass
    ## edit the transfer fees / types

    transfer_types = ['loan' if "oan" in f else 'transfer' for f in transfer_fees]


    transfer_fees_new = []

    for t in range(len(transfer_fees)):

        if transfer_fees[t] == "-":
            transfer_fees_new.append(0)

        elif transfer_fees[t] == "Loan":
            transfer_fees_new.append(0)

        elif transfer_fees[t] == "End of loan":
            transfer_fees_new.append(0)

        elif transfer_fees[t] == "Free transfer":
            transfer_fees_new.append(0)

        elif "Loan fee:" in transfer_fees[t]:
            if "m" in transfer_fees[t]:
                transfer_fees_new.append( int( float( transfer_fees[t].replace("Loan fee:", "").replace("€", "").replace("m", "") ) * 1000000 ) )
            else:
                transfer_fees_new.append( int( transfer_fees[t].replace("Loan fee:", "").replace("€", "").replace("k", "") ) * 1000 )

        elif transfer_fees[t] == "?":
            transfer_fees_new.append(market_values[t])

        elif "m" in transfer_fees[t]:
            transfer_fees_new.append( int( float( transfer_fees[t].replace("€", "").replace("m", "") ) * 1000000 ) )

        elif "k" in transfer_fees[t]:
            transfer_fees_new.append( int( transfer_fees[t].replace("Loan fee:", "").replace("€", "").replace("k", "") ) * 1000 )

        else:
            transfer_fees_new.append(transfer_fees[t])


    ## check internal

    internal_transfer = []

    for t in range(len(transfered_from)):
        if remove_youth(transfered_from[t]) == remove_youth(transferred_to[t]):
            internal_transfer.append("internal")
        else:
            internal_transfer.append("external")

    country_from = country_from[:-1]

    DOB = None

    for row in pageSoup.select('tr'):
        try:
            if row.select('th')[0].get_text().strip() == "Date of birth:":
                DOB = row.select('td')[0].get_text().strip()
        except:
            pass

    # age at transfer
    if DOB != None:
        DOB = DOB.replace(" Happy Birthday", "")
        year_of_birth = int(DOB[len(DOB)-4:])
        month_of_birth = month_to_number(DOB.split(" ")[0])
        day_of_birth = int(DOB.split(" ")[1].split(",")[0])
        DOB = datetime.date(year_of_birth, month_of_birth, day_of_birth)

    age_at_transfer = [calculate_age_at_transfer(DOB, f) for f in transfer_dates]

    player_view = pd.DataFrame(
    {'transfered_from': transfered_from,
     'transferred_to': transferred_to,
     'market_values': market_values,
     'transfer_fees': transfer_fees_new,
     'transfer_dates': transfer_dates,
     'transfer_season': transfer_season,
     'country_to': country_to,
     'country_from': country_from,
     'transfer_types': transfer_types,
     'internal_external_transfer': internal_transfer,
     'age_at_transfer': age_at_transfer
    })

    player_view['player_id'] = player_id


    ####  ADD YOUTH CLUBS ######################################################
    youth_clubs = None
    try:
        for box in pageSoup.select('div.box'):
            if box.select('div')[0].get_text().strip() == 'Youth clubs':
                youth_clubs = box.select('div')[1].get_text().strip()
    except:
        pass

    youth_clubs_list__ = []

    if youth_clubs != None:

        youth_club = youth_clubs

        for f in youth_club.split(","):
    #             print(f.split(" (")[0].strip().lower(), remove_youth(f.split(" (")[0].strip().lower()))
            youth_clubs_list__.append(remove_youth(f.split(" (")[0].strip().lower()))

    if len(player_view[player_view['age_at_transfer'] <= 18].transfered_from) > 0:
        youth_clubs_list__ = youth_clubs_list__ + [remove_youth(ff) for ff in player_view[player_view['age_at_transfer'] <= 18].transfered_from]

    if len(youth_clubs_list__) > 0:
        player_view['all_youth_clubs'] = ','.join(map(str, list(set(youth_clubs_list__))))
    #         print(','.join(map(str, list(set(youth_clubs_list__)))) )
    else:
        player_view['all_youth_clubs'] = remove_youth(player_view.tail(1).iloc[0]['transfered_from'])

    return(player_view)


def performance_history_pull(base_url, player_id, player_dob):
    # print(base_url.replace("profil", "leistungsdatendetails") + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1")
    pageSoup2 = get_souped_page(base_url.replace("profil", "leistungsdatendetails") + "/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1")

    perf_table = pageSoup2.select('table.items')[0]

    ## set empty lists for collection
    season_list = []
    competition_list = []
    competition_code_list = []
    club_list = []
    in_squad_list = []
    appearances_list = []
    ppg_list = []
    goals_list = []
    assists_list = []
    subbed_on_list = []
    subbed_off_list = []
    yellow_card_list = []
    second_yellow_card_list = []
    red_card_list = []
    penalties_list = []
    mins_played_list = []
    clean_sheets_list = []
    goals_conceded_list = []

    #check position
    if len(perf_table.select('tr')[2].select('td')) == 17:
        position_profile = "GK"
    else:
        position_profile = "OUTFIELD"

    if position_profile == "OUTFIELD":
        for row in perf_table.select('tr')[2::]:
            season = row.select('td')[0].get_text()
            competition = row.select('td')[2].get_text()
            competition_code = row.select('td')[2].select('a')[0]['href'].split("/")[4]
            club = row.select('td')[3].select('img')[0]['alt']
            in_squad = int(row.select('td')[4].get_text())
            appearances = row.select('td')[5].get_text()
            ppg = row.select('td')[6].get_text()
            goals = row.select('td')[7].get_text()
            assists = row.select('td')[8].get_text()
            subbed_on = row.select('td')[10].get_text()
            subbed_off = row.select('td')[11].get_text()
            yellow_card = row.select('td')[12].get_text()
            second_yellow_card = row.select('td')[13].get_text()
            red_card = row.select('td')[14].get_text()
            penalties = row.select('td')[15].get_text()
            mins_played = row.select('td')[17].get_text()

            appearances = 0 if appearances == "-" else int(appearances)
            ppg = 0 if ppg in ["-", "0,00"] else float(ppg)
            goals = 0 if goals == "-" else int(goals)
            assists = 0 if assists == "-" else int(assists)
            subbed_on = 0 if subbed_on == "-" else int(subbed_on)
            subbed_off = 0 if subbed_off == "-" else int(subbed_off)
            yellow_card = 0 if yellow_card == "-" else int(yellow_card)
            second_yellow_card = 0 if second_yellow_card == "-" else int(second_yellow_card)
            red_card = 0 if red_card == "-" else int(red_card)
            penalties = 0 if penalties == "-" else int(penalties)

            if mins_played == "-":
                mins_played = 0
            elif "." in mins_played:
                # print(mins_played)
                mins_played = mins_played.replace("'", "").replace(".", "")
            else:
                mins_played = int(mins_played.replace("'", ""))

            goals_conceded = 0
            clean_sheets = 0

            ## append values
            season_list.append(season)
            competition_list.append(competition)
            competition_code_list.append(competition_code)
            club_list.append(club)
            in_squad_list.append(in_squad)
            appearances_list.append(appearances)
            ppg_list.append(ppg)
            goals_list.append(goals)
            assists_list.append(assists)
            subbed_on_list.append(subbed_on)
            subbed_off_list.append(subbed_off)
            yellow_card_list.append(yellow_card)
            second_yellow_card_list.append(second_yellow_card)
            red_card_list.append(red_card)
            penalties_list.append(penalties)
            mins_played_list.append(mins_played)
            clean_sheets_list.append(clean_sheets)
            goals_conceded_list.append(goals_conceded)

    else:
        for row in perf_table.select('tr')[2::]:
            season = row.select('td')[0].get_text()
            competition = row.select('td')[2].get_text()
            competition_code = row.select('td')[2].select('a')[0]['href'].split("/")[4]
            club = row.select('td')[3].select('img')[0]['alt']
            in_squad = int(row.select('td')[4].get_text())
            appearances = row.select('td')[5].get_text()
            ppg = row.select('td')[6].get_text()
            goals = row.select('td')[7].get_text()
            subbed_on = row.select('td')[9].get_text()
            subbed_off = row.select('td')[10].get_text()
            yellow_card = row.select('td')[11].get_text()
            second_yellow_card = row.select('td')[12].get_text()
            red_card = row.select('td')[13].get_text()
            goals_conceded = row.select('td')[14].get_text()
            clean_sheets = row.select('td')[15].get_text()
            mins_played = row.select('td')[16].get_text()

            appearances = 0 if appearances == "-" else int(appearances)
            ppg = 0 if ppg in ["-", "0,00"] else float(ppg)
            goals = 0 if goals == "-" else int(goals)
            subbed_on = 0 if subbed_on == "-" else int(subbed_on)
            subbed_off = 0 if subbed_off == "-" else int(subbed_off)
            yellow_card = 0 if yellow_card == "-" else int(yellow_card)
            second_yellow_card = 0 if second_yellow_card == "-" else int(second_yellow_card)
            red_card = 0 if red_card == "-" else int(red_card)
            clean_sheets = 0 if clean_sheets == "-" else int(clean_sheets)
            goals_conceded = 0 if goals_conceded == "-" else int(goals_conceded)

            if mins_played == "-":
                mins_played = 0
            elif "." in mins_played:
                # print(mins_played)
                mins_played = mins_played.replace("'", "").replace(".", "")
            else:
                mins_played = int(mins_played.replace("'", ""))

            assists = 0
            penalties = 0

            ## append values
            season_list.append(season)
            competition_list.append(competition)
            competition_code_list.append(competition_code)
            club_list.append(club)
            in_squad_list.append(in_squad)
            appearances_list.append(appearances)
            ppg_list.append(ppg)
            goals_list.append(goals)
            assists_list.append(assists)
            subbed_on_list.append(subbed_on)
            subbed_off_list.append(subbed_off)
            yellow_card_list.append(yellow_card)
            second_yellow_card_list.append(second_yellow_card)
            red_card_list.append(red_card)
            penalties_list.append(penalties)
            mins_played_list.append(mins_played)
            clean_sheets_list.append(clean_sheets)
            goals_conceded_list.append(goals_conceded)

    performance_data = pd.DataFrame(
            {'season': season_list,
            'competition': competition_list,
            'competition_code': competition_code_list,
            'club': club_list,
            'in_squad': in_squad_list,
            'appearances': appearances_list,
            'ppg': ppg_list,
            'goals': goals_list,
            'assists': assists_list,
            'subbed_on': subbed_on_list,
            'subbed_off': subbed_off_list,
            'yellow_card': yellow_card_list,
            'second_yellow_card': second_yellow_card_list,
            'red_card': red_card_list,
            'penalties': penalties_list,
            'mins_played': mins_played_list,
            'clean_sheets': clean_sheets_list,
            'goals_conceded': goals_conceded_list
            })


    performance_data['player_id'] = player_id

    age = []
    for s in performance_data.season:

        if "/" in s:
            year = int(s.split("/")[0])
            if year < 30:
                year = 2000 + year
            else:
                year = 1900 + year
            competition_start_date = datetime.date(year, 8, 1)

        else:
            year = int(year)
            competition_start_date = datetime.date(year, 4, 1)

        age.append(calculate_age(player_dob, competition_start_date))

    performance_data['age'] = age

    return(performance_data)

def market_value_historic_pull(base_url, player_id):

    mv_soup = get_souped_page(base_url.replace("profil", "marktwertverlauf"))

    if mv_soup.find("script", text=re.compile("Highcharts.Chart")) != None:

        script = mv_soup.find("script", text=re.compile("Highcharts.Chart")).text
        parsed = js2xml.parse(script)


        xpath = '//array//object//property'

        age_list = []
        club_list = []
        mv_list = []
        date_of_value_list = []

        for i in range(len(parsed.xpath(xpath))):

            age = None
            club = None
            raw_value = None
            date_of_value = None
            date_raw = None

            if parsed.xpath(xpath)[i].get('name') == 'age':
                age = int(stringify_children(parsed.xpath(xpath)[i]).split("number value=")[1].split("/")[0][1:][:-1])
                age_list.append(age)

            if parsed.xpath(xpath)[i].get('name') == 'verein':
                club = stringify_children(parsed.xpath(xpath)[i]).split("<string>")[1].split("</string>")[0].lower()
                club_list.append(club)

            if parsed.xpath(xpath)[i].get('name') == 'mw':
                raw_value = stringify_children(parsed.xpath(xpath)[i]).split("<string>")[1].split("</string>")[0].replace("€", "")

                if "m" in raw_value:
                    raw_value = int( float(raw_value.strip().replace("â‚¬", "").replace("€","").replace("m","")) * 1000000 )
                elif "Th." in raw_value:
                    raw_value = int(raw_value.strip().replace("â‚¬", "").replace("€","").replace("Th.","")) * 1000
                elif "-":
                    raw_value = 0

                mv_list.append(raw_value)

            if parsed.xpath(xpath)[i].get('name') == 'datum_mw':
                date_raw = stringify_children(parsed.xpath(xpath)[i]).split("<string>")[1].split("</string>")[0]
                if date_raw != None:
                    year_of_birth = int(date_raw[len(date_raw)-4:])
                    month_of_birth = month_to_number(date_raw.split(" ")[0])
                    day_of_birth = int(date_raw.split(" ")[1].split(",")[0])
                    date_of_value = datetime.date(year_of_birth, month_of_birth, day_of_birth)

                date_of_value_list.append(date_of_value)


        market_value_history = pd.DataFrame(
        {'club': club_list,
         'value': mv_list,
         'data_date': date_of_value_list,
         'age': age_list
        })

        market_value_history['player_id'] = player_id

        return(market_value_history)

    else:
        market_value_history = pd.DataFrame(
        {'club': [None],
         'value': [None],
         'data_date': [None],
         'age': [None]
        })

        market_value_history['player_id'] = player_id

        return(market_value_history)




def tm_pull(player_page,
            data_folder = "",
            player_bio = False,
            player_status = False,
            transfer_history = False,
            performance_data = False,
            market_value_history = False,
            output = "pandas" ):

    player_id = player_page.split("/")[-1:][0]
    raw_base_page = get_souped_page(player_page)

    bio = bio_player_pull(raw_base_page, player_id)
    output_dict = {}

    player_dob = bio['dob']

    ### if the user has selected to pull player_bio data then run
    if player_bio:

        # pandas output
        if output == "pandas":
            output_dict['player_bio'] = pd.DataFrame.from_dict(bio, orient = "index").transpose()

        # csv output
        elif output == "csv":
            with open((data_folder + player_id + '_bio.csv'),'w') as f:
                w = csv.writer(f)
                w.writerow(bio.keys())
                w.writerow(bio.values())

    ### if the user has selected to pull player_status data then run
    if player_status:

        status = current_football_bio_player_pull(raw_base_page, player_id)

        # pandas output
        if output == "pandas":
            output_dict['player_status'] = pd.DataFrame.from_dict(status, orient = "index").transpose()

        # csv output
        elif output == "csv":
            with open((data_folder + player_id + '_status.csv'),'w') as f:
                w = csv.writer(f)
                w.writerow(status.keys())
                w.writerow(status.values())

    ### if the user has selected to pull player_transfers data then run
    if transfer_history:

        transfers = transfer_history_pull(raw_base_page, player_id)

        # pandas output
        if output == "pandas":
            output_dict['transfer_history'] = transfers

        # csv output
        elif output == "csv":
            transfers.to_csv((data_folder + player_id + '_transfer_history.csv'), index = False)


    ### if the user has selected to pull market_value_history data then run
    if market_value_history:

        historic_market_value = market_value_historic_pull(player_page, player_id)

        # pandas output
        if output == "pandas":
            output_dict['market_value_history'] = historic_market_value

        # csv output
        elif output == "csv":
            historic_market_value.to_csv((data_folder + player_id + '_historic_market_value.csv'), index = False)


    ### if the user has selected to pull performance data then run
    if performance_data:

        perf_data = performance_history_pull(player_page, player_id, player_dob)

        # pandas output
        if output == "pandas":
            output_dict['performance_data'] = perf_data

        # csv output
        elif output == "csv":
            perf_data.to_csv((data_folder + player_id + '_performance_data.csv'), index = False)

    # return all pandas output
    if output == "pandas":
        return(output_dict)

def squad_number_history(base_url, squad_type = "both"):
    '''
    This method returns the squad numbers history of a player.

    Squad numbers history is returned according to the squad type. If the squad type is both,
    then the squad numbers of the players representation for both club and country is returned.

    Args:
    -----------
        base_url: transfermarkt url of the player, string
        squad_type: The type of squad. Default - both. Can be one of ["club", "country", "both], string
    
    Returns:
    -----------
        squad_df: History of squad numbers of the player, Pandas DataFrame
    
    Raises:
    -----------
        Only User Warnings are raised when the squad type doesn't match with one of the 3 options and
        if the player hasn't played for his country and the user requested for both squad numbers.
    '''

    # Check if the squad type is in one of the 3 options
    if squad_type not in ["club", "country", "both"]:
        warnings.warn("Unsupported squad type", UserWarning)
        return None
    
    #Get player id
    player_id = base_url.split("/")[-1:][0]

    # Scrape the squad number page.
    souped_page = get_souped_page(base_url.replace("profil", "rueckennummern"))
    
    #Get the tables with the data directly.
    tables = souped_page.findAll("table", {"class": "items"})

    # Get the columns and add a user defined column
    columns = [col.get_text() for col in tables[0].findAll("th") if col.get_text() != ""]
    columns.append("squad_type")

    # Checking if the player has played for his country
    if len(tables) == 1:
        isNationalAvailable = False
    else:
        isNationalAvailable = True
    values = []

    #Parsing the club squad numbers
    if squad_type in ["both", "club"]:
        for row in tables[0].findAll("tr"):
            row_elem = [val.get_text() for val in row.findAll("td") if val.get_text() != ""]
            if len(row_elem) > 0:
                row_elem.append("club")
                values.append(row_elem)

    #Parsing the coutry squad numbers
    if squad_type in ["both", "country"] and isNationalAvailable:
        for row in tables[1].findAll("tr"):
            row_elem = [val.get_text() for val in row.findAll("td") if val.get_text() != ""]
            if len(row_elem) > 0:
                row_elem.append("country")
                values.append(row_elem)
    elif isNationalAvailable!=True and squad_type in ["both", "country"]:
        warnings.warn("Player hasn't played for his country", UserWarning)
    
    # pandas dataframe output
    if len(values) == 0:
        return None
    squad_df = pd.DataFrame(values, columns=columns)
    squad_df["player_id"] = player_id
    return squad_df

### Create Directory Structure

In [11]:
# Make the data directory structure
for folder in lst_folders:
    path = os.path.join(data_dir_tm, folder)
    if not os.path.exists(path):
        os.mkdir(path)
        for data_types in lst_data_types:
            path = os.path.join(data_dir_tm, folder, data_types)
            if not os.path.exists(path):
                os.mkdir(path)
                os.mkdir(os.path.join(path, 'archive'))
                for league in lst_league_codes:
                    path = os.path.join(data_dir_tm, folder, data_types, league)
                    if not os.path.exists(path):
                        os.mkdir(path)
                        for season in lst_seasons:
                            path = os.path.join(data_dir_tm, folder, data_types, league, season)
                            if not os.path.exists(path):
                                os.mkdir(path)
                                os.mkdir(os.path.join(path, 'archive'))

### Notebook Settings

In [12]:
# Display all columns of pandas DataFrames
pd.set_option('display.max_columns', None)

---

<a id='section2'></a>

## <a id='#section2'>2. Project Brief</a>
This Jupyter notebook is part of a series of notebooks, to scrape, parse, engineer, and unify datasets, that can be used for modeling purposes.

This particular notebook is one of several web scraping notebooks, that scrapes player bio and status (valuation) data from [TransferMarkt](https://www.transfermarkt.co.uk/) using [Beautifulsoup](https://pypi.org/project/beautifulsoup4/) and the [Tyrone Mings web scraper](https://github.com/FCrSTATS/tyrone_mings) by [FCrSTATS](https://twitter.com/FC_rstats). This landed data is then manipulated as DataFrames using [pandas](http://pandas.pydata.org/).

[TransferMarkt](https://www.transfermarkt.co.uk/) is a German-based website owned by [Axel Springer](https://www.axelspringer.com/en/) and is the leading website for the football transfer market. The website posts football related data, including: scores and results, football news, transfer rumours, and most usefully for us - calculated estimates ofthe market values for teams and individual players.

To read more about how these estimations are made, [Beyond crowd judgments: Data-driven estimation of market value in association football](https://www.sciencedirect.com/science/article/pii/S0377221717304332) by Oliver Müllera, Alexander Simons, and Markus Weinmann does an excellent job of explaining how the estimations are made and their level of accuracy.

This notebook, along with the other notebooks in this project workflow are shown in the following diagram:

![roadmap](../../img/football_analytics_data_roadmap.png)

Links to these notebooks in the [`football_analytics`](https://github.com/eddwebster/football_analytics) GitHub repository can be found at the following:
*    [1. Webscraping](https://github.com/eddwebster/football_analytics/tree/master/notebooks/1_data_scraping)
     +    [FBref Player Stats Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/FBref%20Player%20Stats%20Web%20Scraping.ipynb)
     +    [TransferMarket Player Bio and Status Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Bio%20and%20Status%20Web%20Scraping.ipynb)
     +    [TransferMarket Player Valuation Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Valuation%20Web%20Scraping.ipynb)
     +    [TransferMarkt Player Recorded Transfer Fees Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Webscraping.ipynb)
     +    [Capology Player Salary Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/Capology%20Player%20Salary%20Web%20Scraping.ipynb)
     +    [FBref Team Stats Webscraping](https://github.com/eddwebster/football_analytics/blob/master/notebooks/1_data_scraping/FBref%20Team%20Stats%20Web%20Scraping.ipynb)
*    [2. Data Parsing](https://github.com/eddwebster/football_analytics/tree/master/notebooks/2_data_parsing)
     +    [ELO Team Ratings Data Parsing](https://github.com/eddwebster/football_analytics/blob/master/notebooks/2_data_parsing/ELO%20Team%20Ratings%20Data%20Parsing.ipynb)
*    [3. Data Engineering](https://github.com/eddwebster/football_analytics/tree/master/notebooks/3_data_engineering)
     +    [FBref Player Stats Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/FBref%20Player%20Stats%20Data%20Engineering.ipynb)
     +    [TransferMarket Player Bio and Status Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Bio%20and%20Status%20Data%20Engineering.ipynb)
     +    [TransferMarket Player Valuation Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Valuation%20Data%20Engineering.ipynb)
     +    [TransferMarkt Player Recorded Transfer Fees Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Data%20Engineering.ipynb)
     +    [Capology Player Salary Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Player%20Salary%20Data%20Engineering.ipynb)
     +    [FBref Team Stats Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/FBref%20Team%20Stats%20Data%20Engineering.ipynb)
     +    [ELO Team Ratings Data Parsing](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/ELO%20Team%20Ratings%20Data%20Parsing.ipynb)
     +    [TransferMarkt Team Recorded Transfer Fee Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Team%20Recorded%20Transfer%20Fee%20Data%20Engineering.ipynb) (aggregated from [TransferMarkt Player Recorded Transfer Fees notebook](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Recorded%20Transfer%20Fees%20Data%20Engineering.ipynb))
     +    [Capology Team Salary Data Engineering](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Team%20Salary%20Data%20Engineering.ipynb) (aggregated from [Capology Player Salary notebook](https://github.com/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/Capology%20Player%20Salary%20Data%20Engineering.ipynb))
*    [4. Data Unification](https://github.com/eddwebster/football_analytics/tree/master/notebooks/4_data_unification)
*    [5. Modeling and Data Analysis]()
     +    [FBref Player Similarity (PCA and K-Means Clustering)]()

---

<a id='section3'></a>

## <a id='#section3'>3. Data Sources</a>

<a id='section3.3.1'></a>

### <a id='#section3.1'>3.1. Introduction</a>
Before conducting our EDA, the data needs to be imported as a DataFrame in the Data Sources section ([Section 3](#section3)) and cleaned in the Data Engineering section ([Section 4](#section4)).

We'll be using the [pandas](http://pandas.pydata.org/) library to import our data to this workbook as a DataFrame.

<a id='section3.3.2'></a>

### <a id='#section3.2'>3.2. Data Dictionaries</a>
The [TransferMarkt](https://www.transfermarkt.co.uk/) dataset has six features (columns) with the following definitions and data types:

| Feature     | Data type    |
|------|-----|
| `position_number`    | object     |
| `position_description`    | object     |
| `name`    | object     |
| `dob`    | object     |
| `nationality`    | object     |
| `value`    | object     |

<a id='section3.3.3'></a>

### <a id='#section3.3'>3.3. Creating the DataFrame - scraping the data</a>

<a id='section3.3.1'></a>

#### <a id='#section3.3.1.'>3.3.1. League Codes</a>
Before scraping data from [TransferMarkt](https://www.transfermarkt.co.uk/), we need to look at the leagues that we wish to scrape, using the [Tyrone Mings web scraper](https://github.com/FCrSTATS/tyrone_mings) by [FCrSTATS](https://twitter.com/FC_rstats).

The [Tyrone Mings web scraper](https://github.com/FCrSTATS/tyrone_mings) is made up of two parts:
1.    In the first part, the scraper takes the webpages for each of the individual leagues  e.g. the Premier League, and extract the hyperlinks to the pages of all the individual teams in the league table.
2.    In the second part the script, the webscraper uses the list of invidual teams hyperlinks collected in part 1 to then collect the hyperlinks for each of the players for those teams. From this, the scraper can then extract the information we need for each of these players. This information is downloaded in two parts - bio and status. This is then joined together using the player_id.

This information collected for all the players is converted to a [pandas](http://pandas.pydata.org/) DataFrame from which we can view and manipulate the data.

An example webpage for a football league is the following: https://www.transfermarkt.co.uk/jumplist/startseite/wettbewerb/GB1/plus/?saison_id=2019. As we can see, between the subdirectory path of `'/wettbewerb/'` and the `'/plus/'`, there is a 3 or 4 digit code. For The Premier League, the code is GB1.

The following leagues are all the first division and major world leagues, that feature in the publicly available datasets available, including the [FBref](https://fbref.com/en/) 'Big 5' European leagues and MLS dataset, the [FIFA 21 player dataset](https://www.kaggle.com/stefanoleone992/fifa-21-complete-player-dataset), the [Understat](https://understat.com/) 'Big 5' European leagues and Russian league dataset, and [Capology](https://www.capology.com/) player salary datasets, to name but a few. This [TransferMarkt](https://www.transfermarkt.co.uk/) data will be subsequently matched to these datasets.

| League Name               | Region   | Country              | [TransferMarkt](https://www.transfermarkt.co.uk/) League Code     |
|----------------------------|----------|----------------------|-------------|
| Girabola                   | Africa   | Angola               | AN1L        |
| Liga Profesional           | Americas | Argentina            | AR1N        |
| A-League                   | Asia     | Australia            | AUS1        |
| Bundesliga                 | Europe   | Austria              | A1          |
| Bangladesh PL              | Asia     | Bangladesh           | BGD1        |
| Jupiler Pro League         | Europe   | Belgium              | BE1         |
| Div. Prof. Apertura        | Americas | Bolivia              | BO1A        |
| Série A                    | Americas | Brazil               | BRA1        |
| CanPL                      | Americas | Canada               | CDN1        |
| Primera División           | Americas | Chile                | CLPD        |
| Super League               | Asia     | China                | CSL         |
| Liga Dimayor I             | Americas | Colombia             | COLP        |
| Liga DIMAYOR II            | Americas | Colombia             | COL1        |
| Primera Div. Apertura      | Americas | Costa Rica           | CRPD        |
| Primera Div. Clausura      | Americas | Costa Rica           | PDV1        |
| 1.HNL                      | Europe   | Croatia              | KR1         |
| Fortuna Liga               | Europe   | Czech Republic       | TS1         |
| Superligaen                | Europe   | Denmark              | DK1         |
| Serie A Primera Etapa      | Americas | Ecuador              | EL1A        |
| Serie A Segunda Etapa      | Americas | Ecuador              | EL1S        |
| Primera División Apertura  | Americas | El Salvador          | SL1A        |
| Premier League             | Europe   | England              | GB1         |
| Championship               | Europe   | England              | GB2         |
| League One                 | Europe   | England              | GB3         |
| League Two                 | Europe   | England              | GB4         |
| Premier League             | Asia     | Fiji                 | FIJ1        |
| Ligue 1                    | Europe   | France               | FR1         |
| Ligue 2                    | Europe   | France               | FR2         |
| Bundesliga                 | Europe   | Germany              | L1          |
| 2. Bundesliga              | Europe   | Germany              | L2          |
| 3. Liga                    | Europe   | Germany              | L3          |
| Premier League             | Africa   | Ghana                | GHPL        |
| Super League 1             | Europe   | Greece               | GR1         |
| Liga Nacional Apertura     | Americas | Guatemala            | GU1A        |
| Liga Nacional Clausura     | Americas | Guatemala            | GU1C        |
| Hong Kong Premier League   | Asia     | Hongkong             | HGKG        |
| NB I.                      | Europe   | Hungary              | UNG1        |
| Indian Super League        | Asia     | India                | IND1        |
| Liga 1 Indonesia           | Asia     | Indonesia            | IN1L        |
| Persian Gulf Pro League    | Asia     | Iran                 | IRN1        |
| First Division             | Europe   | Ireland              | IR1         |
| Ligat ha'Al                | Europe   | Israel               | ISR1        |
| Serie A                    | Europe   | Italy                | IT1         |
| Serie B                    | Europe   | Italy                | IT2         |
| Jamaica Premier League     | Americas | Jamaica              | JPL1        |
| J1 League                  | Asia     | Japan                | JAP1        |
| K League 1                 | Asia     | Korea, South         | RSK1        |
| BC Olimp Premier League    | Asia     | Kyrgyzstan           | KG1L        |
| Super League               | Asia     | Malaysia             | MYS1        |
| Liga MX Apertura           | Americas | Mexico               | MEXA        |
| Coupe du Trône Marocaine   | Africa   | Morocco              | MARP        |
| Mo√ßambola                 | Africa   | Mozambique           | MO1L        |
| National League            | Asia     | Myanmar              | MYA1        |
| Nepal Super League         | Asia     | Nepal                | NPL1        |
| Eredivisie                 | Europe   | Netherlands          | NL1         |
| NZ Central League          | Asia     | New Zealand          | NCL1        |
| NZ Northern League         | Asia     | New Zealand          | NNL1        |
| Eliteserien                | Europe   | Norway               | NO1         |
| Liga Panamená Apertura     | Americas | Panama               | PN1A        |
| Liga Panamená Clausura     | Americas | Panama               | PN1C        |
| Primera División Apertura  | Americas | Paraguay             | PR1A        |
| Primera División Clausura  | Americas | Paraguay             | PR1C        |
| Liga 1 Apertura            | Americas | Peru                 | TDeA        |
| Liga 1 Clausura            | Americas | Peru                 | TDeC        |
| PFL                        | Asia     | Philippines          | PFL1        |
| Ekstraklasa                | Europe   | Poland               | PL1         |
| Liga Bwin                  | Europe   | Portugal             | PO1         |
| Stars League               | Asia     | Qatar                | QSL         |
| Liga 1                     | Europe   | Romania              | RO1         |
| Premier Liga               | Europe   | Russia               | RU1         |
| Professional League        | Asia     | Saudi Arabia         | SA1         |
| Premiership                | Europe   | Scotland             | SC1         |
| Super liga Srbije          | Europe   | Serbia               | SER1        |
| Premier League             | Asia     | Singapore            | SIN1        |
| DStv Premiership           | Africa   | South Africa         | SFA1        |
| LaLiga                     | Europe   | Spain                | ES1         |
| LaLiga2                    | Europe   | Spain                | ES2         |
| Allsvenskan                | Europe   | Sweden               | SE1         |
| Super League               | Europe   | Switzerland          | C1          |
| Vysshaya Liga              | Asia     | Tajikistan           | TAD1        |
| Thai League                | Asia     | Thailand             | THA1        |
| Ligue I Pro                | Africa   | Tunisia              | TUN1        |
| Tunesischer Supercup       | Africa   | Tunisia              | TUSC        |
| Süper Lig                 | Europe   | Turkey               | TR1         |
| Premier Liga               | Europe   | Ukraine              | UKR1        |
| UAE Gulf League            | Asia     | United Arab Emirates | UAE1        |
| MLS                        | Americas | United States        | MLS1        |
| Primera División Apertura  | Americas | Uruguay              | URU1        |
| Superliga                  | Asia     | Uzbekistan           | UZ1         |
| Primera División           | Americas | Venezuela            | VZ1L        |
| V.League 1                 | Asia     | Vietnam              | VIE1        |

<a id='section3.3.2'></a>

#### <a id='#section3.3.2.'>3.3.2. Define Season and Leagues to Scrape</a>

##### Leagues

In [13]:
# Read in reference data

## Read in data as pandas DataFrame
df_leagues = pd.read_csv(data_dir_tm + '/reference/tm_leagues_comps.csv')

## Display DataFrame
df_leagues.head()

,league_name,region,country,league_code
0,Girabola,Africa,Angola,AN1L
1,Liga Profesional,Americas,Argentina,AR1N
2,A-League,Asia,Australia,AUS1
3,Bundesliga,Europe,Austria,A1
4,Bangladesh PL,Asia,Bangladesh,BGD1


In [14]:
# Convert 'league_code' column of 'df_leagues' DataFrame to a list
lst_leagues_codes = df_leagues['league_code'].to_numpy().tolist()

##### Seasons

In [15]:
# Define season to scrape

## Assign season to variable
season = '2021'    # '2021' for the 21/22 season

## Create 'Full Season' and 'Short Season' strings

### Full season
full_season_string = str(int(season)) + '/' + str(int(season) + 1)

### Short season
short_season_string = str((str(int(season))[-2:]) + (str(int(season) + 1)[-2:]))

In [16]:
full_season_string

'2021/2022'

In [17]:
short_season_string

'2122'

<a id='section3.3.3'></a>

#### <a id='#section3.3.3.'>3.3.3. Player URLs</a>

In [18]:
# Define function to download TransferMarkt Player URLs for the all leagues (lst_leagues) and seasons (lst_seasons) of interest
def get_player_urls(lst_league_codes, lst_seasons):
    
    """
    Function to scrape Player Urls from TransferMarkt for selected leagues and seasons, saves as CSV files, 
    unify scraped CSV files into a single DataFrame that is also subsequently saved as a CSV file,
    and convert final DataFrame to a list or URLs.
    
    Function takes two variables - lst_league_codes, lst_seasons. By default, the lst_league_codes variables is 
    a list for all leagues code in the reference DataFrame and the lst_seasons variable is a list for
    '2016', '2017', '2018', '2019', '2020', and '2021' seasons.
    """
    
    ## Create list of league codes from reference 'df_leagues' DataFrame
    leagues  = lst_league_codes
    
    
    ## Define seasons to scrape
    seasons = lst_seasons
        
    
    ## Save Player URL List (if not already saved)
    if not os.path.exists(os.path.join(data_dir_tm + f'/raw/player_urls/tm_player_urls_combined.csv')):
    
    
        ### Start timer
        tic = datetime.datetime.now()


        ### Print time scraping started
        print(f'Scraping started at: {tic}')


        ### Scrape information for each player
        for season in lst_seasons:

            #### Print message
            print(f'Scraping started for the {season} season...')

            #### Loop through leagues
            for league in lst_league_codes:

                ##### Save Player URL List (if not already saved)
                if not os.path.exists(os.path.join(data_dir_tm + f'/raw/player_urls/{league}/{season}/tm_player_urls_{league}_{season}.csv')):

                    ###### Print message
                    print(f'Scraping started for the {league} league of the {season} season...')

                    ###### Create empty list
                    lst_player_season_urls = []

                    ###### Try
                    try:
                        lst_output = get_player_urls_from_league_page(f'https://www.transfermarkt.co.uk/championship/startseite/wettbewerb/{league}/plus/?saison_id={season}', verbose=True)
                        lst_player_season_urls.append(lst_output)

                        ####### Flatten nested list into a single list
                        lst_player_season_urls = reduce(lambda x,y: x+y, lst_player_season_urls)

                        ####### Dedupe list
                        lst_player_season_urls = list(dict.fromkeys(lst_player_season_urls))

                        ####### Import DataFrame as a CSV file
                        df_player_urls = pd.DataFrame(lst_player_season_urls, columns=['player_url'])

                        ###### Create columns for league code and season
                        df_player_urls ['league_code'] = league
                        df_player_urls ['season'] = season

                        ####### Map season to DataFrame
                        df_player_urls['season'] = df_player_urls['season'].map(dict_seasons)

                        ####### Join league reference DataFrame
                        df_player_urls = pd.merge(df_player_urls, df_leagues, left_on='league_code', right_on='league_code', how='left')

                        ####### Select columns of interest - reorder columns
                        df_player_urls = df_player_urls[['player_url', 'season', 'league_name', 'region', 'country', 'league_code']]

                        ####### Drop duplicates
                        df_player_urls = df_player_urls.drop_duplicates()

                        ####### Export DataFrame
                        df_player_urls.to_csv(data_dir_tm + f'/raw/player_urls/{league}/{season}/tm_player_urls_{league}_{season}.csv', index=None, header=True)

                        ###### Print statement for league and season
                        print(f'All player URLs for the {league} league for {season} season appended.')

                    ###### Except
                    except:
                        pass

                    
            #### Print statement for season    
            print(f'All player URLs for the {season} season appended.')


            
        ### Load player URL List (if already saved)
        else:

            #### Print statement
            print(f'Player URL already saved as a CSV file.')  


            
        ### End timer
        toc = datetime.datetime.now()


        ### Print time scraping ended
        print(f'Scraping ended at: {toc}')


        ### Calculate time take
        total_time = (toc-tic).total_seconds()
        print(f'Time taken to scrape the player urls for the is: {total_time/60:0.2f} minutes.')
    
    
        ### Unify individual CSV files as a single DataFrame

        #### Show files in directory
        all_files = glob.glob(os.path.join(data_dir_tm + f'/raw/player_urls/*/*/tm_player_urls_*_*.csv'))

        #### Create an empty list of Players URLs
        lst_player_urls_all = []

        #### Loop through list of files and read into temporary DataFrames, appending to Player URL list 
        for filename in all_files:
            df_temp = pd.read_csv(filename, index_col=None, header=0)
            lst_player_urls_all.append(df_temp)

        #### Concatenate the files into a single DataFrame
        df_player_urls_all = pd.concat(lst_player_urls_all, axis=0, ignore_index=True)


        ### Sort DataFrame
        df_player_urls_all = df_player_urls_all.sort_values(['league_code', 'season', 'player_url'], ascending=[True, True, True])



        ### Export DataFrame
        df_player_urls_all.to_csv(data_dir_tm + f'/raw/player_urls/tm_player_urls_combined.csv', index=None, header=True)
    
    
    ## Load player URL List CSV
    
    else:
        
        ### 
        print(f'Player URL already saved as a CSV file.')  
        
        ### 
        df_player_urls_all = pd.read_csv(data_dir_tm + f'/raw/player_urls/tm_player_urls_combined.csv')
    
    
    ## Convert DataFrame of Player URLs into a list
    
    ### pandas DataFrame to column
    lst_player_urls_all = df_player_urls_all['player_url'].tolist()

    ## Dedupe list
    lst_player_urls_all = list(dict.fromkeys(lst_player_urls_all))
    
    ### List length i.e. total players
    #len(lst_player_urls_all)
    
    
    ## Return final list of Player URLs
    return(lst_player_urls_all)

In [23]:
# Call 'lst_player_url' function to scrape player URLs or load previously existing DataFrame
lst_player_urls = get_player_urls(lst_league_codes, ['2016', '2017', '2018', '2019', '2020', '2021'])

Player URL already saved as a CSV file.


In [20]:
len(lst_player_urls)

77203

<a id='section3.3.4'></a>

#### <a id='#section3.3.4.'>3.3.4. Bio Information</a>

In [37]:
# Define function to scrape TransferMarkt Player Bio data for the all leagues (lst_leagues) and seasons (lst_seasons) of interest
def get_player_bio(lst_league_codes, lst_seasons):
    
    """
    Function to...
    """
    
    
    ## Define list of league names
    leagues  = lst_league_codes
    
    
    ## Define seasons to scrape
    seasons = lst_seasons
    
    
    ## Start timer
    tic = datetime.datetime.now()


    ## Print time scraping started
    print(f'Scraping started at: {tic}')


    ## Scrape information for each player
    for season in lst_seasons:

        ### Print message
        print(f'Scraping started for the {season} season...')

        ### Loop through leagues
        for league in leagues:

            #### Save Player URL List (if not already saved)
            if not os.path.exists(data_dir_tm + f'/raw/bio/{league}/{season}/tm_player_bio_{league}_{season}_latest.csv'):

                ##### Open DataFrame of Player URLs
                df_player_urls = pd.read_csv(data_dir_tm + f'/raw/player_urls/{league}/{season}/tm_player_urls_{league}_{season}.csv')
                
                ##### Create a list of URLs from saved DataFrame
                lst_player_urls = df_player_urls['player_url'].to_numpy().tolist()
                
                ##### Print message
                print(f'Scraping started for bio data for the {league} league of the {season} season...')

                ##### Create an empty list for scraped player bio data to be appended
                lst_df_bio = []

                ##### Scrape bio information for each player in player url list
                for player_page in lst_player_urls:
                    try:
                        dict_output = tm_pull(player_page, player_bio=True, output='pandas')
                        time.sleep(1)
                        df = dict_output['player_bio']
                        lst_df_bio.append(df)
                        print(f'Bio data appended for: {player_page}')
                    except:
                        print(f'Unable to append Bio data for: {player_page}')
                        pass                        

                ##### Concatenate list into a pandas DataFrame
                df_bio = pd.concat(lst_df_bio)
                
                ###### Create columns for league code and season
                df_bio['league_code'] = league
                df_bio['season'] = season
                
                ###### Drop duplicates
                df_bio = df_bio.drop_duplicates()
                        
                ##### Save DataFrame
                df_bio.to_csv(data_dir_tm + f'/raw/bio/{league}/{season}/tm_player_bio_{league}_{season}_latest.csv', index=None, header=True)        
                
                ##### Export a copy to the 'archive' subfolder, including the date
                df_bio.to_csv(data_dir_tm + f'/raw/bio/{league}/{season}/archive/tm_player_bio_{league}_{season}_last_updated_{today}.csv', index=None, header=True)        
                
                ###### Print statement for league and season
                print(f'All Player Bio data for the {league} league for {season} season scraped and saved.')
                        
            #### Load bio data (if already saved)
            else:

                ##### Print statement
                print(f'Player Bio data for the {league} league for {season} season already saved as a CSV file.')             

    ## End timer
    toc = datetime.datetime.now()
    
    
    ## Print time scraping ended
    print(f'Scraping ended at: {toc}')

    
    ## Calculate time take
    total_time = (toc-tic).total_seconds()
    print(f'Time taken to scrape the Bio data for {len(leagues)} leagues for {len(seasons)} seasons is: {total_time/60:0.2f} minutes.')

    
    ## Unify individual CSV files as a single DataFrame
    
    ### Show files in directory
    all_files = glob.glob(os.path.join(data_dir_tm + f'/raw/bio/*/*/tm_player_bio_*_*_latest.csv'))
    
    ### Create an empty list of Players URLs
    lst_player_bio_all = []

    ### Loop through list of files and read into temporary DataFrames, appending to Player URL list 
    for filename in all_files:
        df_temp = pd.read_csv(filename, index_col=None, header=0)
        lst_player_bio_all.append(df_temp)

    ### Concatenate the files into a single DataFrame
    df_player_bio_all = pd.concat(lst_player_bio_all, axis=0, ignore_index=True)
    
    
    ## Sort DataFrame
    df_player_bio_all = df_player_bio_all.sort_values(['league_code', 'season', 'player_id'], ascending=[True, True, True])

    
    
    ## Export DataFrame
    
    ###
    df_player_bio_all.to_csv(data_dir_tm + f'/raw/bio/tm_player_bio_combined_latest.csv', index=None, header=True)
    
    ###
    df_player_bio_all.to_csv(data_dir_tm + f'/raw/bio/archive/tm_player_bio_combined_last_updated_{today}.csv', index=None, header=True)
    
    ## Distinct number of TransferMarkt IDs i.e. total players
    total_player = df_player_bio_all['player_id'].nunique()


    ## Print statement
    print(f'Player Bio DataFrame contains {total_player} players.')
    
    
    ## Return final list of Player URLs
    return(df_player_bio_all)

In [22]:
# Call 'get_player_bio' data function and create a DataFrame
df_player_bio = get_player_bio(lst_big5_mls_league_codes, lst_seasons)

Scraping started at: 2021-08-28 12:37:09.776197
Scraping started for the 2016 season...
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Scraping started for the 2017 season...
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Player Bio data already saved as a CSV file.
Scraping started for the 2018 season...
Scraping started for bio data for the GB1 league of the 2018 season...
Bio data appended for: https://www.transfermarkt.com/sead-kolasinac/profil/spieler/94005
Bio data appended for: https://www.transfermarkt.com/sokratis-papastathopoulos/profil/spieler/34322
Bio data appended for: https://w

Bio data appended for: https://www.transfermarkt.com/emerson/profil/spieler/181778
Bio data appended for: https://www.transfermarkt.com/cesar-azpilicueta/profil/spieler/57500
Bio data appended for: https://www.transfermarkt.com/jorginho/profil/spieler/102017
Bio data appended for: https://www.transfermarkt.com/danny-drinkwater/profil/spieler/73491
Bio data appended for: https://www.transfermarkt.com/antonio-rudiger/profil/spieler/86202
Bio data appended for: https://www.transfermarkt.com/lucas-piazon/profil/spieler/176485
Bio data appended for: https://www.transfermarkt.com/eden-hazard/profil/spieler/50202
Bio data appended for: https://www.transfermarkt.com/kenedy/profil/spieler/281404
Bio data appended for: https://www.transfermarkt.com/ethan-ampadu/profil/spieler/392771
Bio data appended for: https://www.transfermarkt.com/olivier-giroud/profil/spieler/82442
Bio data appended for: https://www.transfermarkt.com/abdul-rahman-baba/profil/spieler/224884
Bio data appended for: https://www

Bio data appended for: https://www.transfermarkt.com/maxime-le-marchand/profil/spieler/60558
Bio data appended for: https://www.transfermarkt.com/cyrus-christie/profil/spieler/158623
Bio data appended for: https://www.transfermarkt.com/havard-nordtveit/profil/spieler/42234
Bio data appended for: https://www.transfermarkt.com/matt-oriley/profil/spieler/406634
Bio data appended for: https://www.transfermarkt.com/andre-schurrle/profil/spieler/58205
Bio data appended for: https://www.transfermarkt.com/luca-de-la-torre/profil/spieler/315762
Bio data appended for: https://www.transfermarkt.com/ryan-sessegnon/profil/spieler/392775
Bio data appended for: https://www.transfermarkt.com/floyd-ayite/profil/spieler/77974
Bio data appended for: https://www.transfermarkt.com/fabri/profil/spieler/45882
Bio data appended for: https://www.transfermarkt.com/aleksandar-mitrovic/profil/spieler/51152
Bio data appended for: https://www.transfermarkt.com/joe-bryan/profil/spieler/194460
Bio data appended for: 

Bio data appended for: https://www.transfermarkt.com/hamza-choudhury/profil/spieler/341501
Bio data appended for: https://www.transfermarkt.com/filip-benkovic/profil/spieler/293168
Bio data appended for: https://www.transfermarkt.com/marc-albrighton/profil/spieler/61560
Bio data appended for: https://www.transfermarkt.com/leonardo-ulloa/profil/spieler/54890
Bio data appended for: https://www.transfermarkt.com/callum-elder/profil/spieler/271997
Bio data appended for: https://www.transfermarkt.com/bartosz-kapustka/profil/spieler/246579
Bio data appended for: https://www.transfermarkt.com/harry-maguire/profil/spieler/177907
Bio data appended for: https://www.transfermarkt.com/tom-heaton/profil/spieler/34130
Bio data appended for: https://www.transfermarkt.com/stephen-ward/profil/spieler/34691
Bio data appended for: https://www.transfermarkt.com/ashley-westwood/profil/spieler/91317
Bio data appended for: https://www.transfermarkt.com/anthony-driscoll-glennon/profil/spieler/503683
Bio data 

Bio data appended for: https://www.transfermarkt.com/florent-hadergjonaj/profil/spieler/238809
Bio data appended for: https://www.transfermarkt.com/juninho-bacuna/profil/spieler/348863
Bio data appended for: https://www.transfermarkt.com/rajiv-van-la-parra/profil/spieler/77001
Bio data appended for: https://www.transfermarkt.com/aaron-mooy/profil/spieler/123951
Bio data appended for: https://www.transfermarkt.com/joel-coleman/profil/spieler/285551
Bio data appended for: https://www.transfermarkt.com/erik-durm/profil/spieler/93922
Bio data appended for: https://www.transfermarkt.com/jonas-lossl/profil/spieler/48870
Bio data appended for: https://www.transfermarkt.com/collin-quaner/profil/spieler/123397
Bio data appended for: https://www.transfermarkt.com/ramadan-sobhi/profil/spieler/312017
Bio data appended for: https://www.transfermarkt.com/demeaco-duhaney/profil/spieler/380913
Bio data appended for: https://www.transfermarkt.com/ben-hamer/profil/spieler/67283
Bio data appended for: ht

...

Bio data appended for: https://www.transfermarkt.com/cristian-tello/profil/spieler/141834
Bio data appended for: https://www.transfermarkt.com/andres-guardado/profil/spieler/20506
Bio data appended for: https://www.transfermarkt.com/nabil-fekir/profil/spieler/203496
Bio data appended for: https://www.transfermarkt.com/william-carvalho/profil/spieler/100131
Bio data appended for: https://www.transfermarkt.com/paul-akouokou/profil/spieler/429901
Bio data appended for: https://www.transfermarkt.com/edgar-gonzalez/profil/spieler/401624
Bio data appended for: https://www.transfermarkt.com/loren-moron/profil/spieler/354741
Bio data appended for: https://www.transfermarkt.com/german-pezzella/profil/spieler/113133
Bio data appended for: https://www.transfermarkt.com/martin-montoya/profil/spieler/68645
Bio data appended for: https://www.transfermarkt.com/rodri-sanchez/profil/spieler/630995
Bio data appended for: https://www.transfermarkt.com/rui-silva/profil/spieler/234811
Bio data appended for

Bio data appended for: https://www.transfermarkt.com/mauro-arambarri/profil/spieler/290249
Bio data appended for: https://www.transfermarkt.com/jakub-jankto/profil/spieler/319160
Bio data appended for: https://www.transfermarkt.com/ignasi-miquel/profil/spieler/70245
Bio data appended for: https://www.transfermarkt.com/jaime-mata/profil/spieler/227961
Bio data appended for: https://www.transfermarkt.com/djene/profil/spieler/221150
Bio data appended for: https://www.transfermarkt.com/hugo-duro/profil/spieler/573775
Bio data appended for: https://www.transfermarkt.com/josete-miranda/profil/spieler/355336
Bio data appended for: https://www.transfermarkt.com/chema/profil/spieler/248330
Bio data appended for: https://www.transfermarkt.com/enes-unal/profil/spieler/251106
Bio data appended for: https://www.transfermarkt.com/jose-macias/profil/spieler/546047
Bio data appended for: https://www.transfermarkt.com/filip-malbasic/profil/spieler/162652
Bio data appended for: https://www.transfermarkt

Bio data appended for: https://www.transfermarkt.com/giorgi-mamardashvili/profil/spieler/502676
Bio data appended for: https://www.transfermarkt.com/maxi-gomez/profil/spieler/396894
Bio data appended for: https://www.transfermarkt.com/uros-racic/profil/spieler/417575
Bio data appended for: https://www.transfermarkt.com/manu-vallejo/profil/spieler/527350
Bio data appended for: https://www.transfermarkt.com/thierry-correia/profil/spieler/367466
Bio data appended for: https://www.transfermarkt.com/daniel-wass/profil/spieler/63463
Bio data appended for: https://www.transfermarkt.com/gabriel-paulista/profil/spieler/149498
Bio data appended for: https://www.transfermarkt.com/denis-cheryshev/profil/spieler/98322
Bio data appended for: https://www.transfermarkt.com/toni-lato/profil/spieler/320271
Bio data appended for: https://www.transfermarkt.com/omar-alderete/profil/spieler/353032
Bio data appended for: https://www.transfermarkt.com/koba-koindredi/profil/spieler/569375
Bio data appended for

Bio data appended for: https://www.transfermarkt.com/stefan-savic/profil/spieler/107010
Bio data appended for: https://www.transfermarkt.com/angel-correa/profil/spieler/266807
Bio data appended for: https://www.transfermarkt.com/ivan-saponjic/profil/spieler/274347
Bio data appended for: https://www.transfermarkt.com/santiago-arias/profil/spieler/120443
Bio data appended for: https://www.transfermarkt.com/jan-oblak/profil/spieler/121483
Bio data appended for: https://www.transfermarkt.com/mario-hermoso/profil/spieler/281769
Bio data appended for: https://www.transfermarkt.com/rodrigo-de-paul/profil/spieler/255901
Bio data appended for: https://www.transfermarkt.com/felipe/profil/spieler/156501
Bio data appended for: https://www.transfermarkt.com/koke/profil/spieler/74229
Bio data appended for: https://www.transfermarkt.com/renan-lodi/profil/spieler/476352
Bio data appended for: https://www.transfermarkt.com/thomas-lemar/profil/spieler/205562
Bio data appended for: https://www.transferma

FileNotFoundError: [Errno 2] No such file or directory: '../../data/tm/raw/player_urls/MLS1/2021/tm_player_urls_MLS1_2021.csv'

In [34]:
# Display DataFrame
df_player_bio_all.head()

,player_id,player_name,day_of_birth,month_of_birth,year_of_birth,pob,cob,dob,position,height,foot,citizenship,second_citizenship,league_code,season
19991,2963,martín demichelis,20.0,12.0,1980.0,Justiniano Posse,Argentina,1980-12-20,Defender - Centre-Back,184.0,right,NaN,Italy,ES1,2016
20075,5404,jérémy mathieu,29.0,10.0,1983.0,Luxeuil-les-Bains,France,1983-10-29,Defender - Centre-Back,189.0,left,France,NaN,ES1,2016
20567,5856,daniele bonera,31.0,5.0,1981.0,Brescia,Italy,1981-05-31,Defender - Centre-Back,183.0,right,Italy,NaN,ES1,2016
20624,7593,fernando navarro,25.0,6.0,1982.0,Barcelona,Spain,1982-06-25,Defender - Left-Back,176.0,left,Spain,NaN,ES1,2016
20088,7600,andrés iniesta,11.0,5.0,1984.0,Fuentealbilla,Spain,1984-05-11,midfield - Central Midfield,171.0,both,Spain,NaN,ES1,2016


In [35]:
df_player_bio_all.shape

(24211, 15)

<a id='section3.3.5'></a>

#### <a id='#section3.3.5.'>3.3.5. Status Information</a>

In [28]:
# Define function to scrape TransferMarkt Player Status data for the all leagues (lst_leagues) and seasons (lst_seasons) of interest
def get_player_status(lst_league_codes, lst_seasons):
    
    """
    Function to...
    """
    
    ## Define list of league names
    leagues  = lst_league_codes
    
    
    ## Define seasons to scrape
    seasons = lst_seasons
    
    
    ## Start timer
    tic = datetime.datetime.now()


    ## Print time scraping started
    print(f'Scraping started at: {tic}')


    ## Scrape information for each player
    for season in lst_seasons:

        ### Print message
        print(f'Scraping started for the {season} season...')

        ### Loop through leagues
        for league in leagues:

            #### Save Player URL List (if not already saved)
            if not os.path.exists(data_dir_tm + f'/raw/status/{league}/{season}/tm_player_status_{league}_{season}_latest.csv'):

                ##### Open DataFrame of Player URLs
                df_player_urls = pd.read_csv(data_dir_tm + f'/raw/player_urls/{league}/{season}/tm_player_urls_{league}_{season}.csv')
                
                ##### Create a list of URLs from saved DataFrame
                lst_player_urls = df_player_urls['player_url'].to_numpy().tolist()
                
                ##### Print message
                print(f'Scraping started for status data for the {league} league of the {season} season...')

                ##### Create an empty list for scraped player status data to be appended
                lst_df_status = []

                ##### Scrape status information for each player in player url list
                for player_page in lst_player_urls:
                    try:
                        dict_output = tm_pull(player_page, player_status=True, output='pandas')
                        time.sleep(1)
                        df = dict_output['player_status']
                        lst_df_status.append(df)
                        print(f'Status data appended for: {player_page}')
                    except:
                        print(f'Unable to append status data for: {player_page}')
                        pass                        

                ##### Concatenate list into a pandas DataFrame
                df_status = pd.concat(lst_df_status)
                
                ###### Create columns for league code and season
                df_status['league_code'] = league
                df_status['season'] = season
                
                ###### Drop duplicates
                df_status = df_status.drop_duplicates()
                        
                ##### Save DataFrame
                df_status.to_csv(data_dir_tm + f'/raw/status/{league}/{season}/tm_player_status_{league}_{season}_latest.csv', index=None, header=True)        
                
                ##### Export a copy to the 'archive' subfolder, including the date
                df_status.to_csv(data_dir_tm + f'/raw/status/{league}/{season}/archive/tm_player_status_{league}_{season}_last_updated_{today}.csv', index=None, header=True)        
                
                ###### Print statement for league and season
                print(f'All Player Status data for the {league} league for {season} season scraped and saved.')
                        
            #### Load status data (if already saved)
            else:

                ##### Print statement
                print(f'Player Status data for the {league} league for {season} season already saved as a CSV file.')                  

    ## End timer
    toc = datetime.datetime.now()
    
    
    ## Print time scraping ended
    print(f'Scraping ended at: {toc}')

    
    ## Calculate time take
    total_time = (toc-tic).total_seconds()
    print(f'Time taken to scrape the Status data for {len(leagues)} leagues for {len(seasons)} seasons is: {total_time/60:0.2f} minutes.')

    
    ## Unify individual CSV files as a single DataFrame
    
    ### Show files in directory
    all_files = glob.glob(os.path.join(data_dir_tm + f'/raw/status/*/*/tm_player_status_latest.csv'))
    
    ### Create an empty list of Players URLs
    lst_player_status_all = []

    ### Loop through list of files and read into temporary DataFrames, appending to Player URL list 
    for filename in all_files:
        df_temp = pd.read_csv(filename, index_col=None, header=0)
        lst_player_status_all.append(df_temp)

    ### Concatenate the files into a single DataFrame
    df_player_status_all = pd.concat(lst_player_status_all, axis=0, ignore_index=True)
    
    
    ## Sort DataFrame
    df_player_status_all = df_player_status_all.sort_values(['league_code', 'season', 'player_id'], ascending=[True, True, True])

    
    
    ## Export DataFrame
    
    ###
    df_player_status_all.to_csv(data_dir_tm + f'/raw/status/tm_player_status_combined_latest.csv', index=None, header=True)
    
    ###
    df_player_status_all.to_csv(data_dir_tm + f'/raw/status/archive/tm_player_status_combined_last_updated_{today}.csv', index=None, header=True)
    
    ## Distinct number of TransferMarkt IDs i.e. total players
    total_player = df_status['player_id'].nunique()


    ## Print statement
    print(f'Player Status DataFrame contains {total_player} players.')
    
    
    ## Return final list of Player URLs
    return(df_player_status_all)

In [ ]:
# 
df_player_status = get_player_status(lst_league_codes, lst_seasons)

In [ ]:
# Display DataFrame
df_status.head()

In [ ]:
df_status.shape

___

<a id='section4'></a>

## <a id='#section4'>4. Summary</a>
This notebook scrapes data from [TransferMarkt](https://www.transfermarkt.co.uk/) using [Beautifulsoup](https://pypi.org/project/beautifulsoup4/) and the [Tyrone Mings web scraper](https://github.com/FCrSTATS/tyrone_mings) by [FCrSTATS](https://twitter.com/FC_rstats). This landed data is then manipulated as DataFrames using [pandas](http://pandas.pydata.org/).

This data includes Bio and Status data, as well as Transfer History, Performance History and Market Value History data.

___

<a id='section5'></a>

## <a id='#section5'>5. Next Steps</a>
This data is now ready to be engineered before being matched to other datasets such as data from [FBref](https://fbref.com/) and [Capology](https://www.capology.com/).

The Data Engineering subfolder in GitHub can be found [here](https://github.com/eddwebster/football_analytics/tree/master/notebooks/B\)%20Data%20Engineering) and a static version of the FBref data engineering notebookecord can be found [here](https://nbviewer.org/github/eddwebster/football_analytics/blob/master/notebooks/3_data_engineering/TransferMarkt%20Player%20Bio%20and%20Status%20Data%20Engineering.ipynb).

___

<a id='section6'></a>

## <a id='#section6'>6. References</a>

#### Data and Web Scraping
*    [tyrone_mings GitHub repository](https://github.com/FCrSTATS/tyrone_mings) by [FCrSTATS](https://github.com/FCrSTATS)
*    [Python Package Index (PyPI) tyrone-mings library](https://pypi.org/project/tyrone-mings/)
*    [Beyond crowd judgments: Data-driven estimation of market value in association football](https://www.sciencedirect.com/science/article/pii/S0377221717304332) by Oliver Müllera, Alexander Simons, and Markus Weinmann.
*    [06/04/2020: BBC - Premier League squads 'drop £1.6bn in value'](https://www.bbc.co.uk/sport/football/52221463).

---

***Visit my website [eddwebster.com](https://www.eddwebster.com) or my [GitHub Repository](https://github.com/eddwebster) for more projects. If you'd like to get in contact, my Twitter handle is [@eddwebster](http://www.twitter.com/eddwebster) and my email is: edd.j.webster@gmail.com.***

[Back to the top](#top)